# LangChain Expression Language (LCEL)

In [1]:
import os
import openai
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv(".env",override=True) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
#!pip install pydantic==1.10.8

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

## Simple Chain

In [15]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model = "gpt-5-nano")
output_parser = StrOutputParser()

In [7]:
chain = prompt | model | output_parser

In [8]:
chain.invoke({"topic": "bears"})

"Why did the bear break up with his girlfriend?\nBecause he couldn't bear the relationship anymore!"

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

In [10]:
lst_of_txt = ["harrison worked at kensho","bears like to eat honey"]

vectorstore = DocArrayInMemorySearch.from_texts(lst_of_txt,embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

C:\Users\CGDC\AppData\Roaming\Python\Python312\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [11]:
retriever.get_relevant_documents("where did harrison work?")

C:\Users\CGDC\AppData\Local\Temp\ipykernel_12940\3310280720.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("where did harrison work?")


[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [ ]:
retriever.get_relevant_documents("what do bears like to eat")

In [18]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [19]:
from langchain.schema.runnable import RunnableMap

In [22]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

chain = inputs | prompt | model | output_parser

In [23]:
chain.invoke({"question": "where did harrison work?"})

'Kensho.'

In [24]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind

and OpenAI Functions

In [27]:
functions = [
    {"type": "function",
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [31]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)

model = ChatOpenAI(model ="gpt-5-nano").bind(functions=functions)

In [32]:
runnable = prompt | model

In [33]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 153, 'prompt_tokens': 143, 'total_tokens': 296, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--6f86ec4d-4da0-46a2-bc2e-aa232e55fa6c-0', usage_metadata={'input_tokens': 143, 'output_tokens': 153, 'total_tokens': 296, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}})

In [34]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [35]:
model = model.bind(functions=functions)

In [36]:
runnable = prompt | model

In [37]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"Patriots"}', 'name': 'sports_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 283, 'prompt_tokens': 176, 'total_tokens': 459, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--ca7a1007-1206-4966-acb8-cb5210f788dd-0', usage_metadata={'input_tokens': 176, 'output_tokens': 283, 'total_tokens': 459, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 256}})

## Fallbacks

In [48]:
from langchain_openai import OpenAI # old model
import json

In [54]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="gpt-3.5-turbo-instruct"
)
simple_chain = simple_model | json.loads

In [55]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [58]:
simple_model.invoke(challenge)

'\n\n{\n    "title": "Autumn Leaves",\n    "author": "Emily Dickinson",\n    "first_line": "The leaves are falling, one by one"\n}\n\n{\n    "title": "The Ocean\'s Song",\n    "author": "Pablo Neruda",\n    "first_line": "I hear the ocean\'s song, a symphony of waves"\n}\n\n{\n    "title": "A Winter\'s Night",\n    "author": "Robert Frost",\n    "first_line": "The snow falls softly, covering the ground"\n}'

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next line is expected to fail.</p>

In [57]:
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 9 column 1 (char 125)

In [63]:
model = ChatOpenAI(temperature=0) # new model
chain = model | StrOutputParser() | json.loads

In [64]:
chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

In [65]:
final_chain = simple_chain.with_fallbacks([chain])

In [66]:
final_chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers that perches in the soul'}}

## Interface

In [67]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [68]:
chain.invoke({"topic": "bears"})

'Why did the bear break up with his girlfriend? Because she was unbearable!'

In [69]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why don't bears wear socks?\n\nBecause they already have bear feet!",
 'Why are frogs so happy? Because they eat whatever bugs them!']

In [72]:
for t in chain.stream({"topic": "bears"}):
    print(t, end = " ")

 Why  do  bears  have  hairy  coats ?

 F ur  protection !  

In [71]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears wear shoes?\n\nBecause they have bear feet!"